## Import librairies

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
import cv2
from pathlib import Path
import os
from sklearn.model_selection import train_test_split
from datetime import datetime


## Import data

In [65]:
# Charger le CSV
csv_path = '_classes2.csv'
df = pd.read_csv(csv_path)

# Dossier contenant les images
image_path = r"C:\Users\malam\Documents\2 - ECOLE\HENALLUX VIRTON\M1 - INGENIEUR INDUSTRIEL AUTOMATION\13 - SYSTEMES INTELLIGENTS\Self-Driving Cars.v6-version-4-prescan-416x416.multiclass\train"
image_dir = Path(image_path)

# Vérifier les colonnes disponibles (les labels)
label_columns = df.columns.tolist()[1:]  # On exclut 'filename'

print("Colonnes de label :", label_columns)

# Vérifie que le fichier existe
df['filepath'] = df['filename'].apply(lambda x: str(image_dir / x))
df = df[df['filepath'].apply(os.path.exists)]  # Garde uniquement les fichiers valides


Colonnes de label : [' Green Light', ' Red Light', ' Speed Limit 10', ' Speed Limit 100', ' Speed Limit 110', ' Speed Limit 120', ' Speed Limit 20', ' Speed Limit 30', ' Speed Limit 40', ' Speed Limit 50', ' Speed Limit 60', ' Speed Limit 70', ' Speed Limit 80', ' Speed Limit 90', ' Stop']


## Split into train and test

In [67]:
# Paramètres
IMG_SIZE = (128, 128)
BATCH_SIZE = 32

# Fonction pour lire et redimensionner une image
def load_and_preprocess_image(path):
    img = cv2.imread(path)
    img = cv2.resize(img, IMG_SIZE)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    return img / 255.0  # Normalisation

# Appliquer à tout le dataset
X = np.array([load_and_preprocess_image(path) for path in df['filepath']])
y = df[label_columns].values.astype(np.float32)  # Multilabel binaires (0/1)

# Split train/test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Paramètres d'augmentation
ROTATION_RANGE = 30
ZOOM_RANGE = 0.2
WIDTH_SHIFT = 0.2
HEIGHT_SHIFT = 0.2
HORIZONTAL_FLIP = True

# Augmentation uniquement sur le train
train_datagen = ImageDataGenerator(
    rotation_range=ROTATION_RANGE,
    zoom_range=ZOOM_RANGE,
    width_shift_range=WIDTH_SHIFT,
    height_shift_range=HEIGHT_SHIFT,
    horizontal_flip=HORIZONTAL_FLIP
)

train_generator = train_datagen.flow(X_train, y_train, batch_size=BATCH_SIZE)
test_generator = ImageDataGenerator().flow(X_test, y_test, batch_size=BATCH_SIZE)


## Build the model

In [68]:
model = models.Sequential([
    # Couche d'entrée : spécifie la forme des images (128x128 pixels, 3 canaux RGB)
    layers.Input(shape=(*IMG_SIZE, 3)),
    
    # Première couche de convolution :
    # - 32 filtres de taille 3x3
    # - Fonction d'activation ReLU
    layers.Conv2D(16, (3, 3), activation='relu'),
    # Réduction de dimension par max pooling 2x2
    layers.MaxPooling2D(2, 2),
    
    # Deuxième couche de convolution :
    # - 64 filtres (plus de filtres pour détecter plus de caractéristiques)
    layers.Conv2D(32, (3, 3), activation='relu'),
    layers.MaxPooling2D(2, 2),
    
    # Troisième couche de convolution :
    # - 128 filtres pour des caractéristiques plus complexes
    layers.Conv2D(64, (3, 3), activation='relu'),
    layers.MaxPooling2D(2, 2),
    
    # Aplatissement des données pour les couches denses
    layers.Flatten(),
    # Couche dense avec 128 neurones 
    layers.Dense(64, activation='relu'),
    # Dropout de 50% pour éviter le surapprentissage
    layers.Dropout(0.5),
    
    # Couche de sortie :
    # - Autant de neurones que de classes (len(label_columns))
    # - Activation sigmoid pour la classification multi-label
    layers.Dense(len(label_columns), activation='sigmoid')
])


## Train the model

In [69]:
# Compilation et entraînement du modèle
model.compile(
    optimizer='adam',
    loss='binary_crossentropy',  # Pour classification multi-label
    metrics=['accuracy']
)



# Create a timestamped folder for this run
timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
checkpoint_dir = f'model_checkpoints_{timestamp}'
os.makedirs(checkpoint_dir, exist_ok=True)

callbacks = [
    # EarlyStopping(patience=5, restore_best_weights=True),
    ModelCheckpoint(
        os.path.join(checkpoint_dir, 'best_model.h5'), 
        save_best_only=True
    )
]

EPOCHS = 50  # Nombre d'epochs pour l'entraînement initial

# Augmentation du nombre d'epochs à 15 pour permettre un meilleur apprentissage
# - Plus d'epochs = plus d'opportunités pour le modèle d'apprendre
# - La validation_data permet de suivre le surapprentissage
history = model.fit(
    train_generator,
    epochs=EPOCHS, 
    validation_data=test_generator,
    callbacks=callbacks
)


c:\Users\malam\Documents\10 - VENV\SI_Env\Lib\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/50
87/87 ━━━━━━━━━━━━━━━━━━━━ 0s 86ms/step - accuracy: 0.0728 - loss: 0.4168

87/87 ━━━━━━━━━━━━━━━━━━━━ 11s 99ms/step - accuracy: 0.0728 - loss: 0.4162 - val_accuracy: 0.1470 - val_loss: 0.2668
Epoch 2/50
87/87 ━━━━━━━━━━━━━━━━━━━━ 0s 209ms/step - accuracy: 0.1005 - loss: 0.2985

87/87 ━━━━━━━━━━━━━━━━━━━━ 19s 219ms/step - accuracy: 0.1005 - loss: 0.2984 - val_accuracy: 0.2262 - val_loss: 0.2512
Epoch 3/50
87/87 ━━━━━━━━━━━━━━━━━━━━ 0s 199ms/step - accuracy: 0.1348 - loss: 0.2781

87/87 ━━━━━━━━━━━━━━━━━━━━ 18s 208ms/step - accuracy: 0.1349 - loss: 0.2781 - val_accuracy: 0.2450 - val_loss: 0.2373
Epoch 4/50
87/87 ━━━━━━━━━━━━━━━━━━━━ 0s 190ms/step - accuracy: 0.1660 - loss: 0.2649

...

87/87 ━━━━━━━━━━━━━━━━━━━━ 13s 151ms/step - accuracy: 0.2691 - loss: 0.2284 - val_accuracy: 0.3357 - val_loss: 0.2050
Epoch 15/50
87/87 ━━━━━━━━━━━━━━━━━━━━ 10s 112ms/step - accuracy: 0.2916 - loss: 0.2299 - val_accuracy: 0.3415 - val_loss: 0.2062
Epoch 16/50
87/87 ━━━━━━━━━━━━━━━━━━━━ 0s 138ms/step - accuracy: 0.2766 - loss: 0.2269

87/87 ━━━━━━━━━━━━━━━━━━━━ 13s 144ms/step - accuracy: 0.2767 - loss: 0.2269 - val_accuracy: 0.3386 - val_loss: 0.2035
Epoch 17/50
87/87 ━━━━━━━━━━━━━━━━━━━━ 0s 86ms/step - accuracy: 0.2855 - loss: 0.2272

87/87 ━━━━━━━━━━━━━━━━━━━━ 8s 91ms/step - accuracy: 0.2854 - loss: 0.2273 - val_accuracy: 0.3501 - val_loss: 0.2028
Epoch 18/50
87/87 ━━━━━━━━━━━━━━━━━━━━ 0s 88ms/step - accuracy: 0.2617 - loss: 0.2266

87/87 ━━━━━━━━━━━━━━━━━━━━ 8s 94ms/step - accuracy: 0.2619 - loss: 0.2266 - val_accuracy: 0.3559 - val_loss: 0.2010
Epoch 19/50
87/87 ━━━━━━━━━━━━━━━━━━━━ 8s 89ms/step - accuracy: 0.2996 - loss: 0.2214 - val_accuracy: 0.3372 - val_loss: 0.2014
Epoch 20/50
87/87 ━━━━━━━━━━━━━━━━━━━━ 8s 93ms/step - accuracy: 0.2723 - loss: 0.2239 - val_accuracy: 0.3156 - val_loss: 0.2055
Epoch 21/50
54/87 ━━━━━━━━━━━━━━━━━━━━ 2s 85ms/step - accuracy: 0.3096 - loss: 0.2224

KeyboardInterrupt: 

## Evaluation

In [ ]:
loss, accuracy = model.evaluate(test_generator)
print(f"Test Loss: {loss:.4f}, Test Accuracy: {accuracy:.4f}")

# Affichage des courbes
plt.plot(history.history['accuracy'], label='Train Accuracy')
plt.plot(history.history['val_accuracy'], label='Val Accuracy')
plt.legend()
plt.title("Accuracy")
plt.show()

plt.plot(history.history['loss'], label='Train Loss')
plt.plot(history.history['val_loss'], label='Val Loss')
plt.legend()
plt.title("Loss")
plt.show()


## Save and load the model and try with new data

In [ ]:
import pandas as pd
from datetime import datetime
import os

# Prepare data to save
model_data = {
    'Date': datetime.now().strftime("%Y-%m-%d %H:%M:%S"),
    'Image_Size': f"{IMG_SIZE[0]}x{IMG_SIZE[1]}",
    'Batch_Size': BATCH_SIZE,
    'Epochs': EPOCHS,
    'Final_Accuracy': accuracy,
    'Final_Loss': loss,
    'Model_Architecture': str(model.summary()),
    'Data_Augmentation': {
        'rotation_range': ROTATION_RANGE,
        'zoom_range': ZOOM_RANGE,
        'width_shift_range': WIDTH_SHIFT,
        'height_shift_range': HEIGHT_SHIFT,
        'horizontal_flip': HORIZONTAL_FLIP
    }
}

# Convert to DataFrame
df_new = pd.DataFrame([model_data])

# Check if file exists
csv_filename = 'Model_parameters.csv'
if os.path.exists(csv_filename):
    # Append to existing file
    df_new.to_csv(csv_filename, mode='a', header=False, index=False)
else:
    # Create new file
    df_new.to_csv(csv_filename, index=False)

print(f"Model parameters saved to {csv_filename}")

In [ ]:
def save_model_with_timestamp(model, base_folder="model_saves_Panneaux_Scan"):
    # Create folder if it doesn't exist
    os.makedirs(base_folder, exist_ok=True)
    
    # Generate filename with timestamp
    timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
    model_filename = f"sign_classifier_{timestamp}.h5"
    
    # Full path
    model_path = os.path.join(base_folder, model_filename)
    
    # Save the model
    model.save(model_path)
    
    print(f"Model saved to {model_path}")

# Use the function
save_model_with_timestamp(model)


## Save the model bis